In [1]:
import pandas as pd
from parse import PlaceParser
from reader import read_place_desc, read_html_from
from service import PlaceService, SearchEngine
from writer import write_place_to_tsv
from index import preprocess, ConjunctiveIndex

from os import listdir

from os.path import isfile, join
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm
import os
import time


# Data collection
##  1.1. Get the list of places





In [ ]:
with open('most_popular_places.txt', 'w') as file:
    for i in tqdm(range(300)):
        
        url = 'https://www.atlasobscura.com/places?page='+str(i+1)+'&sort=likes_count'
        list_page = requests.get(url)
        list_soup = BeautifulSoup(list_page.text)
        list_places = [x['href'] for x in list_soup.find_all('a', {'class':"content-card content-card-place"})]
        for place in tqdm(list_places):
            file.write('https://www.atlasobscura.com' + str(place))
            file.write('\n')

## 1.2. Crawl places

In [ ]:
f= open('most_popular_places.txt', 'r')
for j, url in enumerate(f):
    
    if j %18 == 0:
        dir_path = f'page{j//18+1}'
        if not os.path.isdir(dir_path):
            os.mkdir(dir_path)
            
    response = requests.get(url.strip())
    with open(f'{dir_path}/{j+1}.html','w') as file:
        print()
        file.write(response.text)
        
    time.sleep(1)

# Parse Pages

In [4]:
htmls = [read_html_from(f'./pages/{file}') for file in listdir('./pages') if isfile(join('./pages', file))]
   
places = [PlaceParser(html).parse() for html in htmls]

tsv_path = write_place_to_tsv('./', places)

names, descriptions = read_place_desc(tsv_path)


# Create Index

In [5]:
index = ConjunctiveIndex()
for name, desc in zip(names.values, descriptions.values):
    desc = preprocess(desc)

    for token in desc:
        index.put(token, name)

# Create Search Engine

In [6]:
place_service = PlaceService()
place_service.load('./places.tsv')

search_engine = SearchEngine(index, place_service)

# Run Search

In [10]:
search_engine.query('the Clock Tower Building')

,name,tags,num_people_visited,num_people_want,desc,short_desc,nearby,address,lat,lon,editors,publication_date,appears_in,related_places,url
1,Clock Tower Building,"astounding timepieces,architectural oddities,a...",205,859,"Opened in 1898, the Clock Tower Building was o...",A historic 1890s architectural masterpiece wit...,"Mmuseumm,African Burial Ground National Monume...","125 Worth St, New York, New York, 10013, Unite...",40.7157,-74.0025,"dlc31723,hrnick,mbison,cait7911,Zia Ahasan",2009-09-18 00:00:00,19 Amazing Clock Towers,"The Temple of Abu Simbel,Jantar Mantar,Prague ...",NaN
